In [63]:
import pymupdf
import pymupdf4llm
import pandas as pd
import glob
import re

In [64]:
all_pdf = glob.glob(r"PDF2\*.pdf")

In [65]:
# pdf_document = r"PDF2\Как я стал core-разработчиком Python в 19 лет _ Хабр.pdf"
# doc = pymupdf.open(pdf_document)
# print("Исходный документ: ", doc.name)
# print("\nКоличество страниц: %i\n\n------------------\n\n" % doc.page_count)
# print(doc.metadata)
# text = []
# for current_page in range(len(doc)):
#     page = doc.load_page(current_page)
#     page_text = page.get_text("text")
#     print("Стр. ", current_page + 1, "\n")
#     text.append(page_text)
#     print(page_text)

In [66]:
text = ''.join(text).replace('\n',' ')
text

'[КАК СТАТЬ АВТОРОМ](https://habr.com/ru/sandbox/start/)  **[vstorozhilov](https://habr.com/ru/users/vstorozhilov/)**  7 часов назад # Три необсуждаемых вопроса о параллельной распределённой обработке данных — чтобы жить стало легче  **Средний** **12 мин** **328** ##### Python*,  Data Engineering*,  Распределённые системы*  Туториал ##### Привет, Хабр! На связи Владимир, техлид в команде разработки ИИ-инструментов в департаменте технологической надёжности одной из крупных компаний. Наша команда помогает делать корпоративные и клиентские сервисы надежнее помощью Data Science. ## Мы помогаем мониторить тысячи подсистем ##### Система, в разработке которой участвует моя команда, предназначена для централизованного мониторинга тысяч отдельных систем, в том числе распределённых географически. Наша группа отвечает за online-модель, подсвечивающую явно аномальные отклонения в показателях мониторинга. С её помощью инженер дежурной смены однозначно определяет, есть ли сейчас проблемы, требующие 

In [67]:
def extract_markdown_text(document) -> str:
    return pymupdf4llm.to_markdown(document)
    

In [68]:
def parse_markdown(document: str):
    print(document)

In [69]:
def parse_article(document):
    doc = pymupdf.open(document)
    md = extract_markdown_text(document)
    lines = md.splitlines()
    
    # 1. Author
    author = None
    for line in lines:
        m = re.match(r"\*\*\[(.+?)\]\(", line)
        if m:
            author = m.group(1)
            break

    # 2. Company (if any)
    company = None
    for line in lines:
        if re.search(r"\d{2}\.\d{2}\.\d{4}", line) and "/" in line:
            company = line.split("/")[-1].strip()
            break

    # 3. Publish date (absolute or relative)
    publish_date = None
    for line in lines:
        m = re.match(r"(\d{2}\.\d{2}\.\d{4},\s*\d{2}:\d{2})", line)
        if m:
            publish_date = m.group(1); break
        m = re.match(r"(\d{1,2}\s+[А-Яа-я]{3}\s+в\s+\d{2}:\d{2})", line)
        if m:
            publish_date = m.group(1); break
        m = re.search(r"(\d+\s+(?:минут[аи]?|часов?|дней?)\s+назад)", line)
        if m:
            publish_date = doc.metadata['creationDate']; break
        
        if publish_date is None:
            publish_date = doc.metadata['creationDate']

    # 4. Description and Rating
    description = None
    rating = None
    for line in lines:
        parts = re.findall(r"\*\*(.+?)\*\*", line)
        # with company: expect 3+ bold parts: desc, time, rating
        if company and len(parts) >= 3 and re.match(r"^\d+K?$", parts[-1]):
            description = parts[0]
            raw = parts[-1]
            rating = int(raw[:-1])*1000 if raw.endswith("K") else int(raw)
            break
        # without company: expect 2 bold parts: time, rating
        if not company and len(parts) >= 2 and re.match(r"^\d+K?$", parts[-1]):
            raw = parts[-1]
            rating = int(raw[:-1])*1000 if raw.endswith("K") else int(raw)
            break

    # 5. Find the tags line (#### or #####) with comma-separated categories
    tags_idx = next(
        (i for i, line in enumerate(lines)
         if re.match(r"^#{4,5}\s*[^#].*\*,", line)),
        None
    )

    # 6. Capture article_text: all non-blank lines after tags_idx until the first '-----'
    article_text = ""
    if tags_idx is not None:
        content_lines = []
        for line in lines[tags_idx + 1:]:
            if line.strip().startswith("-----"):
                break
            if line.strip():
                content_lines.append(line.strip())
        article_text = "\n\n".join(content_lines)

    # 7. Build result dict
    result = {
        "author": author,
        "publish_date": publish_date,
        "rating": rating,
        "article_text": article_text
    }
    if company:
        result["company_name"] = company
        result["description"] = description
    else:
        result["company_name"] = None
        result["description"] = None

    return result

In [70]:
Brak = []
Author = []
PublishDate = []
Rating = []
ArticleText = []
CompanyName = []
CompanyDescription = []
for pdf in all_pdf:
    # try:  
        text = extract_markdown_text(pdf)


        result = parse_article(pdf)
        author = result["author"]
        publish_date = result["publish_date"]
        rating = result["rating"]
        article_text = result["article_text"]
        company_name = result["company_name"]
        company_description = result["description"]

        Author.append(author)
        PublishDate.append(publish_date)
        Rating.append(rating)
        ArticleText.append(article_text)
        CompanyName.append(company_name)
        CompanyDescription.append(company_description)
    # except:
        # Brak.append(pdf)

In [71]:
Brak

[]

In [72]:
from numpy import float32


# df = pd.DataFrame({
# 'Author': pd.Series(dtype=str),
# 'PublishDate': pd.Series(dtype=float32),
# 'DataPublish': pd.Series(dtype=str),
# 'Activity': pd.Series(dtype=str),
# 'TextArticle': pd.Series(dtype=str)
# })

df = pd.DataFrame({
    'Author': pd.Series(dtype=str),
    'PublishDate': pd.Series(dtype=str),
    'Rating': pd.Series(dtype=float32),
    'ArticleText': pd.Series(dtype=str),
    'CompanyName': pd.Series(dtype=str),
    'CompanyDescription': pd.Series(dtype=str)
})

In [73]:
print(Author)
df['Author'] = Author
df['PublishDate'] = PublishDate
df['Rating'] = Rating
df['ArticleText'] = ArticleText
df['CompanyName'] = CompanyName
df['CompanyDescription'] = CompanyDescription

['TraPhro', None, 'technokratiya', None, 'LizzieSimpson', None, 'BabayMazay', 'El_Gato_Grande', 'AnnieBronson', 'manul', 'Guitariz', 'AlfaTeam', None, 'Exosphere', 'empenoso', 'Systems_Education', 'artyomsoft', 'Eclips4', 'erbanovanastasia', None, 'Arnak', 'TourmalineCore', 'Bright_Translate', 'vstorozhilov']


In [74]:
df

,Author,PublishDate,Rating,ArticleText,CompanyName,CompanyDescription
0,TraPhro,D:20250318034441+00'00',NaN,## От аутсайдера-задиры до лидера литографии\n...,None,None
1,None,D:20230320172822+00'00',NaN,,None,None
2,technokratiya,D:20250303175201+00'00',419.0,,None,None
3,None,D:20230320160606+00'00',NaN,,None,None
4,LizzieSimpson,D:20240208175839+00'00',13000.0,,None,None
5,None,D:20230320171750+00'00',NaN,,None,None
6,BabayMazay,11 апр в 12:01,51.0,,None,None
7,El_Gato_Grande,8 апр в 15:08,5000.0,,None,None
8,AnnieBronson,D:20240208175310+00'00',107.0,,None,None
9,manul,"07.03.2025, 08:20",466.0,,Хабр,Простой


In [75]:
# pdf = extract_text_from_pdf(r"PDF2\ASML_ Создание самых сложных машин на Земле _ Хабр.pdf")

In [76]:
# pars_pdf(pdf[0])